## NFL Playoffs Data Pull
Welcome to this Jupyter notebook on pulling NFL playoff result data from profootballreference

### Importing Packages
I use the BeautifulSoup package for working with HTML, and playwright for the async data scraping

In [1]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time
import re
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
# !pip playwright install-deps firefox

### Defining Globals
Here it keeps track of the file paths, as well as the number of playoff teams in each year of NFL history. <br>
The array concatenation is admittedly a little suspicious, but I print the output to better visualize changes I need to make, if any.

In [45]:
DATA_DIR = "sbpgs";
YR_DIR = "leagueYrs";
PLAYOFF_SIZES = [4] * 4 + [8] * 8 + [10] * 4 + [16] + [10] * 8 + [12] * 30 + [14] * 5;
year_sizes = list(zip(PLAYOFF_SIZES,list(range(1966,2024))))
print(year_sizes)

[(4, 1966), (4, 1967), (4, 1968), (4, 1969), (8, 1970), (8, 1971), (8, 1972), (8, 1973), (8, 1974), (8, 1975), (8, 1976), (8, 1977), (10, 1978), (10, 1979), (10, 1980), (10, 1981), (16, 1982), (10, 1983), (10, 1984), (10, 1985), (10, 1986), (10, 1987), (10, 1988), (10, 1989), (10, 1990), (12, 1991), (12, 1992), (12, 1993), (12, 1994), (12, 1995), (12, 1996), (12, 1997), (12, 1998), (12, 1999), (12, 2000), (12, 2001), (12, 2002), (12, 2003), (12, 2004), (12, 2005), (12, 2006), (12, 2007), (12, 2008), (12, 2009), (12, 2010), (12, 2011), (12, 2012), (12, 2013), (12, 2014), (12, 2015), (12, 2016), (12, 2017), (12, 2018), (12, 2019), (12, 2020), (14, 2021), (14, 2022), (14, 2023)]


In [4]:
def playoff_size(year):
    return PLAYOFF_SIZES[year - 1966]

### Function Declarations
This get_html function is heavily inspired by Dataquest, a youtube creator. <br>
Link to video here: https://www.youtube.com/watch?v=o6Ih934hADU

In [44]:
async def get_html(url,selector,sleep =5, retries =3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep *i)
        
        try:
            async with async_playwright() as p:
#                 chromium is open sourced version of chrome
                browser = await p.firefox.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

Saves each page to the computer, name is a naming function

In [6]:
async def savePath(link,directory,name,tag):
    save_path = os.path.join(directory, name(link))
    if not(os.path.exists(save_path)):
        html = await get_html(link, tag);
        with open(save_path, "w+") as f:
            f.write(html)
    else :
        with open(save_path, 'r') as f:
            html = f.read()
    return html

In [7]:
def getYear(url):
    yr = int(url[url.find(".htm")-4:url.find(".htm")]);
    return yr;

Two digit year is for easier nicknaming of teams in visualizations

In [8]:
def twoDigitYear(year):
    return str(year)[-2:]

In [9]:
twoDigitYear(2004)

'04'

These functions look at the url and help name the file path

In [10]:
def getAbbrv(url):
    abbrv = url[url.index("teams/")+6:url.index(".htm")-5];
    return abbrv;

In [11]:
def nameYear(link):
    i = link.find("years")+6;
    return link[i:i+4] + "league.htm";

In [12]:
def nameTeam(link):
    return getAbbrv(link)+str(getYear(link))+".htm";

Searches array of playoff teams, and gives round of playoff exit

In [13]:
def getResult(index,length):
    if index < 0 :
        return 0
    elif index == length-1 :
      return 5
    elif index == length-2 :
      return 4
    elif index >= length-4 :
      return 3
    elif index >= length-8 :
      return 2
    else :
      return 1

In [46]:
def convertResult(index):
    if index <= 0 :
        return "Missed"
    elif index == 5 :
      return "Won SB"
    elif index == 4:
      return "Lost SB"
    elif index == 3:
      return "Lost Title"
    elif index == 2:
      return "Lost Div"
    else :
      return "Lost WC"

Getting the playoff teams

In [14]:
def getYearURL(year):
    base = f"https://www.pro-football-reference.com";
    url = f"{base}/years/{year}/index.htm"
    return url

In [15]:
async def getPlayoffTeamsArr(url):
    base = f"https://www.pro-football-reference.com";
    a_tags = (await findLosers(url)) + (await findWinner(url));
    hrefs = [a["href"]  for a in a_tags];
    teams = [l for l in hrefs if "/teams/" in l];
    finalTeams = [base + t for t in teams]
    return finalTeams;

In [16]:
async def findWinner(url):
    html = BeautifulSoup(await savePath(url, YR_DIR, nameYear,"#div_playoff_results"))
    winner = html.find_all("td",{'data-stat': 'winner'})[-1]
    a_tag = [div.find_all("a") for div in winner]
    return sum(a_tag,[])

In [17]:
async def findLosers(url):
    html = BeautifulSoup(await savePath(url, YR_DIR, nameYear,"#div_playoff_results"))
    losers = html.find_all("td",{'data-stat': 'loser'})
    html.find_all("td",{'data-stat': 'winner'})[-1]
    a_tags = [div.find_all("a") for div in losers]
    return sum(a_tags,[])

Example of calling code below:

In [18]:
losers = await getPlayoffTeamsArr("https://www.pro-football-reference.com/years/2021/index.htm")
# winners = await findWinner("https://www.pro-football-reference.com/years/2021/index.htm")
# losers = losers.append("")
losers

['https://www.pro-football-reference.com/teams/nwe/2021.htm',
 'https://www.pro-football-reference.com/teams/rai/2021.htm',
 'https://www.pro-football-reference.com/teams/pit/2021.htm',
 'https://www.pro-football-reference.com/teams/dal/2021.htm',
 'https://www.pro-football-reference.com/teams/phi/2021.htm',
 'https://www.pro-football-reference.com/teams/crd/2021.htm',
 'https://www.pro-football-reference.com/teams/oti/2021.htm',
 'https://www.pro-football-reference.com/teams/gnb/2021.htm',
 'https://www.pro-football-reference.com/teams/buf/2021.htm',
 'https://www.pro-football-reference.com/teams/tam/2021.htm',
 'https://www.pro-football-reference.com/teams/kan/2021.htm',
 'https://www.pro-football-reference.com/teams/sfo/2021.htm',
 'https://www.pro-football-reference.com/teams/cin/2021.htm',
 'https://www.pro-football-reference.com/teams/ram/2021.htm']

Searches through array of playoff results to find team, and gives the playoff outcome

In [19]:
async def findResult(teamURL,year):
    yearURL = getYearURL(year)
    size = playoff_size(year)
    if year > 2022 or year < 1970 :
        return "undefined"
    arr = await getPlayoffTeamsArr(yearURL)
    index = arr.index(teamURL) if teamURL in arr else -1
    return getResult(index,size)

In [20]:
await findResult("https://www.pro-football-reference.com/teams/det/1970.htm",2024)

'undefined'

## Pulling Data from HTML
Each function pulls a piece of HTML. <br>
In practice, they are all similar.

In [21]:
def getWins(string):
    w = int(string[:string.find("-")])
    return w;

In [22]:
def getGames(string):
    w = getWins(string)
    rest = string[string.find("-") + 1:]
    l = getWins(rest)
    t = int(rest[rest.find("-") + 1:])
    return w + l + t

In [23]:
# sus url manipulation to get the next year
def getNext(url,diff):
    base = url[:url.find(".htm")-5]
    yr = getYear(url)+diff;
    if((yr > 2023) or (yr < 1950)):
        return "";
    return f"{base}/{yr}.htm";

In [24]:
def searchHTML(string,html,div):
    arr = html.find_all(div)
    for i,p in enumerate(arr):
        if str(p).find(string)>0 :
            return i
    return -1

Meta is the HTML section of the pages that I'm pulling from. <br>
I realized I had a lot of code duplication, so I made this nice helper to execute most pulls.

In [25]:
async def searchMeta(url,div,string):
    html = BeautifulSoup(await savePath(url,DATA_DIR,nameTeam,"#meta"))
    index = searchHTML(string,html,div)
    if index < 0 :
        return "undefined"
    return html.find_all(div)[index].getText();

In [26]:
#get record from html
async def getRec(link):
    ret = await searchMeta(link,"p","Record")
    ret = ret[ret.find(":")+2:ret.find(",")]
    return ret

In [27]:
async def getFullName(url):
    html = BeautifulSoup(await savePath(url,DATA_DIR,nameTeam,"#meta"))
    ret = html.find_all("span")[1].getText()
    return ret

In [28]:
def getNickname(year, name):
    return "'" + twoDigitYear(year) + " " + name.split()[-1]

In [29]:
async def getDivision(url):
    ret = await searchMeta(url,"p","Record")
    if ret == "undefined" :
        return ret;
    return ret[ret.find("\t")+1:ret.find("Div")-1]

In [30]:
async def getConf(url):
    ret = await getDivision(url)
    if ret == "undefined" :
        return ret;
    return ret[0:3]

In [31]:
async def getCoach(url):
    ret = await searchMeta(url,"p","Coach")
    if ret == "undefined" :
        return ret;
    return ret[ret.find("\n")+1:ret.find("(")-1]

In [32]:
# can be buggy for older seasons
async def getSBOdds(url):
    ret = await searchMeta(url,"p","Preseason Odds")
    if ret == "undefined" :
        return ret;
    endI = min(len(ret) - 1, ret.find(";"))
    return ret[ret.find("Bowl")+5:endI]

In [33]:
await getSBOdds("https://www.pro-football-reference.com/teams/rai/1980.htm")

'+350'

In [34]:
# can be buggy for older seasons
async def getOverUnder(url):
    ret = await searchMeta(url,"p","O/U:")
    if ret == "undefined" :
        return ret;
    return float(ret[ret.find("O/U:")+5:])

In [35]:
await searchMeta("https://www.pro-football-reference.com/teams/clt/1970.htm","p","Odds")

'undefined'

In [36]:
await getOverUnder("https://www.pro-football-reference.com/teams/kan/2021.htm")

12.5

In [37]:
async def getPFRank(url):
    ret = await searchMeta(url,"p","Points For")
    if ret == "undefined" :
        return ret;
    return ret[ret.find(")")+2:ret.find("of")-3]

In [38]:
async def getPARank(url):
    ret = await searchMeta(url,"p","Points Against")
    if ret == "undefined" :
        return ret;
    return ret[ret.find(")")+2:ret.find("of")-3]

In [39]:
async def getExpRec(url):
    ret = await searchMeta(url,"p","Expected W-L")
    if ret == "undefined" :
        return ret;
    return ret[ret.find(":")+2:]

In [40]:
async def getSRS(url):
    ret = await searchMeta(url,"p","#srs")
    if ret == "undefined" :
        return ret;
    return ret[ret.find(":")+2:ret.find("(")-1]

In [41]:
async def getSOS(url):
    ret = await searchMeta(url,"p","#sos")
    if ret == "undefined" :
        return ret;
    return ret[ret.find("SOS: ")+5:-1]

## Scrape_season function
Executes the calling of the playoff URL

In [42]:
async def scrape_season(season):
    url = getYearURL(season)
    finalTeams = await getPlayoffTeamsArr(url)
    return finalTeams

## Calling the Functions

In [43]:
SEASONS = list(range(1970,2022));
SEASONS = [await scrape_season(yr) for yr in SEASONS]

## Simple Data Frame

Here, I'm trying to create to visualize the odds a team lands in a given playoff round the next season, based on them winnning a given number of playoff rounds in the current season.  Essentially, a markov matrix of with each playoff result as a row.

In [113]:
simple = [];
for arrays in SEASONS:
    for team in arrays:
        year = getYear(team)
        games = getGames(await getRec(team))
        nextGames = getGames(await getRec(getNext(team,+1)))
        wins = getWins(await getRec(team))
        nextWins = getWins(await getRec(getNext(team,+1)))
        fullName = await getFullName(team)
        simple.append([year,
                         getNickname(year,fullName),
                         await findResult(team, year),
                         await findResult(getNext(team,+1), year+1),
                         games,
                         nextGames,
                         wins,
                         nextWins,
                         nextWins - wins])

In [115]:
sf = pd.DataFrame(simple)
sf.columns = ["Year", "Team","Round","Next_Round","Games","NextGames","W","Next_Wins","Diff"]
# sf.columns = ["Year", "Team","Prev_Round","Round","Next_Round","Prev_Wins","W","Next_Wins"]
sf_losers = sf[sf['Round'] == 4]

In [121]:
output_df = sf_losers[["Year","Team","Next_Wins","Diff"]]
output_df
# output_df.to_csv('winsDropOff.csv', index = False, header=True)

,Year,Team,Next_Wins,Diff
6,1970,'70 Cowboys,11,1
14,1971,'71 Dolphins,14,4
22,1972,'72 Redskins,10,-1
30,1973,'73 Vikings,10,-2
38,1974,'74 Vikings,12,2
46,1975,'75 Cowboys,11,1
54,1976,'76 Vikings,9,-2
62,1977,'77 Broncos,10,-2
72,1978,'78 Cowboys,11,-1
82,1979,'79 Rams,11,2


The following output was created on tableau:
![NFL Playoff Results](sbhangover/nflplayoffresults.png)

## Manipulation of Data for Tableau Output

In [124]:
start = [];

# for teams that made the playoffs
for val in range(0, 6):
    df = sf[sf['Round'] == val]
    percents = df['Next_Round'].value_counts(normalize=True).sort_index().tolist()
    start.append(percents)

# Print the collected percentage breakdowns as a list of lists
start = [percent for percent in start if percent]
rounded = [[round(num, 3) for num in sublist] for sublist in start]
mtrx = np.array(rounded)
df = pd.DataFrame(mtrx)
rows = ['Lost WC', 'Lost Div','Lost Title', 'Lost SB','Won SB']
cols = ['Missed', 'Lost WC', 'Lost Div','Lost Title', 'Lost SB','Won SB']
df.index = rows;
df.columns = cols;
print(df)
# df.to_csv('playoffhangover.csv', index=True, header=True)
# print(rounded)

            Missed  Lost WC  Lost Div  Lost Title  Lost SB  Won SB
Lost WC      0.532    0.128     0.192       0.090    0.032   0.026
Lost Div     0.514    0.111     0.149       0.111    0.053   0.062
Lost Title   0.345    0.109     0.200       0.173    0.055   0.118
Lost SB      0.288    0.115     0.288       0.135    0.096   0.077
Won SB       0.308    0.096     0.231       0.135    0.096   0.135


In [45]:
df = sf[sf['Prev_Round'] == 0]
percents = df['Round'].value_counts(normalize=True).sort_index().tolist()
print(percents)

[0.3682170542635659, 0.37209302325581395, 0.1434108527131783, 0.07364341085271318, 0.04263565891472868]


## Detailed Dataframe

In [122]:
allTeams = [];
for arrays in SEASONS:
    for team in arrays:
        year = getYear(team)
        rec = await getRec(team);
#         nRec = await getRec(arr[1])
        allTeams.append([year,
                         await getFullName(arr[0]),
                         await findResult(arr[0], year),
                         getWins(rec),
                         rec,
                         await getExpRec(arr[0]),
                         getGames(rec),
                         await getPFRank(arr[0]),
                         await getPARank(arr[0]),
                         await getSOS(arr[0]),
                         await getCoach(arr[0]),
                         await getSBOdds(arr[0]),
                         await getOverUnder(arr[0]),
                         year + 1,
                         await findResult(arr[1], year+1),
                         getWins(nRec),
                         nRec,
                         await getExpRec(arr[1]),
                         getGames(nRec),
                         await getPFRank(arr[1]),
                         await getPARank(arr[1]),
                         await getSOS(arr[1]),
                         await getCoach(arr[1]),
                         await getSBOdds(arr[1]),
                         await getOverUnder(arr[1]),
                         getAbbrv(arr[0]),
                         await getConf(arr[0]), 
                         await getDivision(arr[0]),
                         await findResult(getNext(arr[0],-1), year-1) # for looking at previous year
                        ])
                            

ValueError: invalid literal for int() with base 10: ''

In [107]:
df = pd.DataFrame(allTeams)

In [108]:
df.columns = ["Year", "Team","Playoff_Rounds_Won","Wins","Full_Record","Expected_Record","Games",
                  "Points_For_Rank", "Points_Against_Rank", "SOS","Coach",
                  "SB_Odds", "Win_Total_Over_Under","Next_Year","NY_Playoff_Rounds_Won","NY_Wins","NY_Full_Record",
                  "NY_Expected_Record", "NY_Games", "NY_Points_For_Rank", "NY_Points_Against_Rank", "NY_SOS",
                  "NY_Coach","NY_SB_Odds", "NY_Win_Total_Over_Under", "Abbrv","Conf","Division","PY_Playoff_Rounds_Won"]
df

,Year,Team,Playoff_Rounds_Won,Wins,Full_Record,Expected_Record,Games,Points_For_Rank,Points_Against_Rank,SOS,...,NY_Points_For_Rank,NY_Points_Against_Rank,NY_SOS,NY_Coach,NY_SB_Odds,NY_Win_Total_Over_Under,Abbrv,Conf,Division,PY_Playoff_Rounds_Won
0,1971,Dallas Cowboys,5,11,11-3-0,11.3-2.7,14,1,7,-3.29,...,10,6,.48,Tom Landry,undefined,undefined,dal,NFC,NFC East,4
1,1971,Miami Dolphins,4,10,10-3-1,11.2-2.8,14,4,3,-2.38,...,1,1,-4.27,Don Shula,undefined,undefined,mia,AFC,AFC East,2
2,1971,Baltimore Colts,3,10,10-4-0,12.2-1.8,14,5,2,-1.93,...,20,11,-1.10,Don McCafferty,undefined,undefined,clt,AFC,AFC East,5
3,1971,San Francisco 49ers,3,9,9-5-0,9.6-4.4,14,9,6,.26,...,4,9,-1.00,Dick Nolan,undefined,undefined,sfo,NFC,NFC West,3
4,1971,Washington Redskins,2,9,9-4-1,9.9-4.1,14,12,4,-1.01,...,7,3,-2.10,George Allen,undefined,undefined,was,NFC,NFC East,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551,2020,Chicago Bears,1,8,8-8-0,8.1-7.9,16,22,14,.11,...,27,22,.45,Matt Nagy,+6600,7.5,chi,NFC,NFC North,-1
552,2020,Pittsburgh Steelers,1,12,12-4-0,10.6-5.4,16,12,3,-1.85,...,21,20,.75,Mike Tomlin,+5000,8.5,pit,AFC,AFC North,-1
553,2020,Washington Football Team,1,7,7-9-0,8.2-7.8,16,25,4,-1.22,...,23,25,1.62,Ron Rivera,+4000,8.5,was,NFC,NFC East,-1
554,2020,Seattle Seahawks,1,12,12-4-0,10.0-6.0,16,8,15,-0.04,...,16,11,.24,Pete Carroll,+2500,9.5,sea,NFC,NFC West,2


## Searching Data frame

In [80]:
# keep only years up until 2022
df2 = df.loc[df['Year'] < 2022]
df2["NY_Playoff_Rounds_Won"] = pd.to_numeric(df2["NY_Playoff_Rounds_Won"])
df2

/var/folders/l4/pc6_sp9x6dv3bjdsjjpz173r0000gn/T/ipykernel_38755/1156874546.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["NY_Playoff_Rounds_Won"] = pd.to_numeric(df2["NY_Playoff_Rounds_Won"])


,Year,Team,Playoff_Rounds_Won,Wins,Full_Record,Expected_Record,Games,Points_For_Rank,Points_Against_Rank,SOS,...,NY_Games,NY_Points_For_Rank,NY_Points_Against_Rank,NY_SOS,NY_Coach,NY_SB_Odds,NY_Win_Total_Over_Under,Abbrv,Conf,Division
0,1970,Baltimore Colts,5,11,11-2-1,9.5-4.5,14,6,7,-5.81,...,14,5,2,-1.93,Don McCafferty,undefined,undefined,clt,AFC,AFC East
1,1970,Dallas Cowboys,4,10,10-4-0,9.4-4.6,14,10,4,1.40,...,14,1,7,-3.29,Tom Landry,undefined,undefined,dal,NFC,NFC East
2,1970,San Francisco 49ers,3,10,10-3-1,9.2-4.8,14,1,14,.43,...,14,9,6,.26,Dick Nolan,undefined,undefined,sfo,NFC,NFC West
3,1970,Oakland Raiders,3,8,8-4-2,7.2-6.8,14,9,19,.53,...,14,2,14,-1.03,John Madden,undefined,undefined,rai,AFC,AFC West
4,1970,Minnesota Vikings,2,12,12-2-0,12.4-1.6,14,3,1,1.36,...,14,18,1,-1.07,Bud Grant,undefined,undefined,min,NFC,NFC Central
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,2021,Philadelphia Eagles,1,9,9-8-0,9.9-7.1,17,12,18,-0.97,...,17,3,8,-1.32,Nick Sirianni,+2440,9.5,phi,NFC,NFC East
574,2021,Dallas Cowboys,1,12,12-5-0,12.2-4.8,17,1,7,-0.18,...,17,4,5,-0.81,Mike McCarthy,+1960,10.5,dal,NFC,NFC East
575,2021,Pittsburgh Steelers,1,9,9-7-1,7.0-10.0,17,21,20,.75,...,17,26,10,1.48,Mike Tomlin,+8000,7.5,pit,AFC,AFC North
576,2021,Las Vegas Raiders,1,10,10-7-0,6.9-10.1,17,18,26,.56,...,17,12,26,-1.11,Josh McDaniels,+3600,8.5,rai,AFC,AFC West


In [140]:
sf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570 entries, 0 to 569
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Year        570 non-null    int64 
 1   Team        570 non-null    object
 2   Prev_Round  570 non-null    int64 
 3   Round       570 non-null    int64 
 4   Next_Round  570 non-null    int64 
 5   Prev_Wins   570 non-null    int64 
 6   W           570 non-null    int64 
 7   Next_Wins   570 non-null    int64 
dtypes: int64(7), object(1)
memory usage: 35.8+ KB


In [139]:
display(sf.groupby("Round").mean())
# df.groupby("Playoff_Rounds_Won").mean(numeric_only=True)

/var/folders/l4/pc6_sp9x6dv3bjdsjjpz173r0000gn/T/ipykernel_38755/802132348.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  display(sf.groupby("Round").mean())


,Year,Prev_Round,Next_Round,Prev_Wins,W,Next_Wins
Round,,,,,,
1,2001.740741,0.370370,0.487654,8.135802,9.882716,8.617284
2,1996.000000,0.872549,0.759804,8.877451,10.563725,8.931373
3,1996.000000,1.245098,1.529412,9.450980,11.196078,9.676471
4,1996.000000,1.450980,1.411765,9.843137,11.941176,9.705882
5,1996.000000,2.039216,1.784314,10.431373,12.196078,10.549020


In [85]:
df2

,Year,Team,Playoff_Rounds_Won,Wins,Full_Record,Expected_Record,Games,Points_For_Rank,Points_Against_Rank,SOS,...,NY_Games,NY_Points_For_Rank,NY_Points_Against_Rank,NY_SOS,NY_Coach,NY_SB_Odds,NY_Win_Total_Over_Under,Abbrv,Conf,Division
0,1970,Baltimore Colts,5,11,11-2-1,9.5-4.5,14,6,7,-5.81,...,14,5,2,-1.93,Don McCafferty,undefined,undefined,clt,AFC,AFC East
1,1970,Dallas Cowboys,4,10,10-4-0,9.4-4.6,14,10,4,1.40,...,14,1,7,-3.29,Tom Landry,undefined,undefined,dal,NFC,NFC East
2,1970,San Francisco 49ers,3,10,10-3-1,9.2-4.8,14,1,14,.43,...,14,9,6,.26,Dick Nolan,undefined,undefined,sfo,NFC,NFC West
3,1970,Oakland Raiders,3,8,8-4-2,7.2-6.8,14,9,19,.53,...,14,2,14,-1.03,John Madden,undefined,undefined,rai,AFC,AFC West
4,1970,Minnesota Vikings,2,12,12-2-0,12.4-1.6,14,3,1,1.36,...,14,18,1,-1.07,Bud Grant,undefined,undefined,min,NFC,NFC Central
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,2021,Philadelphia Eagles,1,9,9-8-0,9.9-7.1,17,12,18,-0.97,...,17,3,8,-1.32,Nick Sirianni,+2440,9.5,phi,NFC,NFC East
574,2021,Dallas Cowboys,1,12,12-5-0,12.2-4.8,17,1,7,-0.18,...,17,4,5,-0.81,Mike McCarthy,+1960,10.5,dal,NFC,NFC East
575,2021,Pittsburgh Steelers,1,9,9-7-1,7.0-10.0,17,21,20,.75,...,17,26,10,1.48,Mike Tomlin,+8000,7.5,pit,AFC,AFC North
576,2021,Las Vegas Raiders,1,10,10-7-0,6.9-10.1,17,18,26,.56,...,17,12,26,-1.11,Josh McDaniels,+3600,8.5,rai,AFC,AFC West


In [187]:
testDF.sort_values("Year", ascending = False)

,Year,Team,Playoff Rounds Won,Wins,Full Record,Expected Record,Games,Points For Rank,Points Against Rank,SOS,...,NY Games,NY Points For Rank,NY Points Against Rank,NY SOS,NY Coach,NY SB Odds,NY Win Total Over-Under,Abbrv,Conf,Division
591,2022,Los Angeles Chargers,1,10,10-7-0,8.7-8.3,17,13,21,-1.33,...,7,9,24,-0.24,Brandon Staley,+2500,9.5,sdg,AFC,AFC West
584,2022,New York Giants,2,9,9-7-1,8.3-8.7,17,15,17,-0.01,...,8,32,22,2.13,Brian Daboll,+4800,7.5,nyg,NFC,NFC East
578,2022,Kansas City Chiefs,5,14,14-3-0,11.4-5.6,17,1,16,-1.23,...,8,12,2,-2.54,Andy Reid,+600,11.5,kan,AFC,AFC West
579,2022,Philadelphia Eagles,4,14,14-3-0,11.6-5.4,17,3,8,-1.32,...,8,3,19,-2.52,Nick Sirianni,+750,11.5,phi,NFC,NFC East
580,2022,San Francisco 49ers,3,13,13-4-0,12.9-4.1,17,6,1,-2.27,...,8,5,5,.93,Kyle Shanahan,+1000,10.5,sfo,NFC,NFC West
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,1970,Dallas Cowboys,4,10,10-4-0,9.4-4.6,14,10,4,1.40,...,14,1,7,-3.29,Tom Landry,undefined,undefined,dal,NFC,NFC East
5,1970,Miami Dolphins,2,10,10-4-0,9.1-4.9,14,11,5,-6.26,...,14,4,3,-2.38,Don Shula,undefined,undefined,mia,AFC,AFC East
6,1970,Detroit Lions,2,10,10-4-0,11.0-3.0,14,2,2,3.69,...,14,3,16,.21,Joe Schmidt,undefined,undefined,det,NFC,NFC Central
7,1970,Cincinnati Bengals,2,8,8-6-0,8.6-5.4,14,7,9,-3.53,...,14,11,9,-1.91,Paul Brown,undefined,undefined,cin,AFC,AFC Central


## Saving Data Frame as csv

In [49]:
from pathlib import Path
root_path = "~/Desktop"
output_path = Path(root_path, 'output4.csv')
# output_file_full_path = ("/","output")
testDF.to_csv(output_path, index = None)